In [1]:
!pip install -qq transformers genomic-benchmarks datasets

     |████████████████████████████████| 4.4 MB 14.8 MB/s 
     |████████████████████████████████| 362 kB 56.1 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.4 MB/s 
     |████████████████████████████████| 596 kB 17.2 MB/s 
     |████████████████████████████████| 2.3 MB 44.7 MB/s 
     |████████████████████████████████| 271 kB 48.6 MB/s 
     |████████████████████████████████| 140 kB 45.8 MB/s 
     |████████████████████████████████| 212 kB 54.0 MB/s 
     |████████████████████████████████| 1.1 MB 18.7 MB/s 
     |████████████████████████████████| 127 kB 46.5 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 61.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 whic

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
### Parameters
MODEL_NAME = "armheb/DNA_bert_6" #Original DNABert model
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = 1

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 1

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_nontata_promoters', 0)]

BATCH_SIZE = 16
ACCUMULATION = 4
LEARNING_RATE = 1e-5
EPOCHS = 4
RUNS = 5

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/DNABERT.csv'

## Download benchmark datasets and tokenizer

In [2]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version, use_cloud_cache=True)

benchmark_root = Path(BENCHMARKS_FOLDER)

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/4 [00:00<?, ?it/s]

Downloading...
From: https://drive.google.com/uc?id=1cpXg0ULuTGF7h1_HTYvc6p8M-ee43t-v
To: /root/.genomic_benchmarks/demo_coding_vs_intergenomic_seqs.zip

  0%|          | 0.00/33.9M [00:00<?, ?B/s]
100%|██████████| 33.9M/33.9M [00:00<00:00, 227MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JW0-eTB-rJXvFcglqBo3pFZi1kyIWC3X
To: /root/.genomic_benchmarks/demo_human_or_worm.zip

  0%|          | 0.00/28.9M [00:00<?, ?B/s]
100%|██████████| 28.9M/28.9M [00:00<00:00, 231MB/s]
Downloading...
From: https://drive.google.com/uc?id=176563cDPQ5Y094WyoSBF02QjoVQhWuCh
To: /root/.genomic_benchmarks/human_enhancers_cohn.zip

  0%|          | 0.00/11.9M [00:00<?, ?B/s]
100%|██████████| 11.9M/11.9M [00:00<00:00, 52.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VdUg0Zu8yfLS6QesBXwGz1PIQrTW3Ze4
To: /root/.genomic_benchmarks/human_nontata_promoters.zip

  0%|          | 0.00/11.8M [00:00<?, ?B/s]
100%|██████████| 11.8M/11.8M [00:00<00:00, 85.9MB/s]


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 501, 1989, 3848, 3089, 56, 212, 835, 3325, 999, 3983, 3629, 2214, 650, 2587, 2142, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [5]:
import pandas as pd
import numpy as np
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric
from random import random, randrange

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2, gradient_accumulation_steps=ACCUMULATION,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, seed=randrange(1,10001), report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))
        outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
        outputs_df.to_csv(OUTPUT_PATH, index=False)



  0%|          | 0/4 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f5e0a288dd0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/100000 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

...

***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/armheb/DNA_bert_6/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2697389de18c4fe8c3497cea35aaf65130fdd59c3ab64cb6b1c2e0632fefdaf0.3a7e1ca237211e6405270f85616f49989aeee994db35f6593a40c7b5081a50d0
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.352575,0.844809,0.841653
1,0.450100,0.327458,0.855103,0.852905
2,0.326200,0.333350,0.852225,0.848002
3,0.300300,0.323518,0.857649,0.855571


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/armheb/DNA_bert_6/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2697389de18c4fe8c3497cea35aaf65130fdd59c3ab64cb6b1c2e0632fefdaf0.3a7e1ca237211e6405270f85616f49989aeee994db35f6593a40c7b5081a50d0
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.339695,0.852668,0.854329
1,0.440200,0.317458,0.861523,0.865325
2,0.321300,0.324659,0.859531,0.857847
3,0.297400,0.318697,0.862187,0.862019


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [6]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.924280,0.925008,1639.1930
1,demo_coding_vs_intergenomic_seqs,0.923240,0.924572,1636.2917
2,demo_coding_vs_intergenomic_seqs,0.924520,0.926032,1631.4838
3,demo_coding_vs_intergenomic_seqs,0.923920,0.925639,1632.8869
4,demo_coding_vs_intergenomic_seqs,0.925280,0.926745,1628.0106
5,demo_human_or_worm,0.965800,0.965358,1630.5597
6,demo_human_or_worm,0.966960,0.966445,1635.4404
7,demo_human_or_worm,0.965280,0.964629,1637.4602
8,demo_human_or_worm,0.966920,0.966484,1634.9699
9,demo_human_or_worm,0.967400,0.967072,1633.4275


In [7]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy                  f1            \
                                      mean       sem      mean       sem   
dataset                                                                    
demo_coding_vs_intergenomic_seqs  0.924248  0.000336  0.925599  0.000381   
demo_human_or_worm                0.966472  0.000398  0.965998  0.000440   
human_enhancers_cohn              0.743092  0.000501  0.741807  0.002386   
human_nontata_promoters           0.861080  0.000874  0.862067  0.001692   

                                 train_runtime            
                                          mean       sem  
dataset                                                   
demo_coding_vs_intergenomic_seqs    1633.57320  1.933281  
demo_human_or_worm                  1634.37154  1.150253  
human_enhancers_cohn                1182.24632  0.942240  
human_nontata_promoters              704.41262  0.567971

In [8]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)